In [0]:
import urllib.request
urllib.request.urlretrieve("https://www.gutenberg.org/files/1661/1661-0.txt" ,"/tmp/sherlock_holmes.txt")
dbutils.fs.mv("file:/tmp/sherlock_holmes.txt","dbfs:/data/sherlock_holmes.txt")

Out[1]: True

In [0]:
#creating spark context and reading the textfile
from pyspark import SparkContext , SparkConf
conf = SparkConf().setAppName("word count")
sc = SparkContext.getOrCreate(conf = conf)
bookrdd = sc.textFile("/data/sherlock_holmes.txt")

In [0]:
#converting all words to lower case
bookrdd = bookrdd.flatMap(lambda x : x.lower().split(" "))

In [0]:
#removing symbols, punctuation and blank spaces
import re
bookrdd = bookrdd.map(lambda x: re.sub(r'[^A-Za-z]', '', x))
bookedd = bookrdd.map(lambda x : x.strip())
bookrdd = bookrdd.filter(lambda x : len(x)!=0)

In [0]:
#removing stop words
from pyspark.ml.feature import StopWordsRemover
r =  StopWordsRemover()
stop_words_list = r.getStopWords()
bookrdd = bookrdd.filter(lambda x : x not in stop_words_list)

In [0]:
#finding number of words and sorting it
bookrdd = bookrdd.map(lambda x : (x,1))
bookrdd = bookrdd.reduceByKey(lambda x,y : x+y)
bookrdd = bookrdd.sortByKey()